<h1><center>Laboratorio 9: Optimización de modelos 💯</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Ignacio Meza, Gabriel Iturra
- Auxiliar: Sebastián Tinoco
- Ayudante: Arturo Lazcano, Angelo Muñoz

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: simón repolt
- Nombre de alumno 2: magdalena de la fuente


### **Link de repositorio de GitHub:** `https://github.com/HexaPulsar/LabMDS`

## Temas a tratar

- Predicción de demanda usando `xgboost`
- Búsqueda del modelo óptimo de clasificación usando `optuna`
- Uso de pipelines.

## Reglas:

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias.
- Pueden usar cualquer material del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Optimizar modelos usando `optuna`
- Recurrir a técnicas de *prunning*
- Forzar el aprendizaje de relaciones entre variables mediante *constraints*
- Fijar un pipeline con un modelo base que luego se irá optimizando.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

# Importamos librerias útiles

In [13]:
!pip install -qq xgboost optuna

# 1. El emprendimiento de Fiu

Tras liderar de manera exitosa la implementación de un proyecto de ciencia de datos para caracterizar los datos generados en Santiago 2023, el misterioso corpóreo **Fiu** se anima y decide levantar su propio negocio de consultoría en machine learning. Tras varias e intensas negociaciones, Fiu logra encontrar su *primera chamba*: predecir la demanda (cantidad de venta) de una famosa productora de bebidas de calibre mundial. Como usted tuvo un rendimiento sobresaliente en el proyecto de caracterización de datos, Fiu lo contrata como *data scientist* de su emprendimiento.

Para este laboratorio deben trabajar con los datos `sales.csv` subidos a u-cursos, el cual contiene una muestra de ventas de la empresa para diferentes productos en un determinado tiempo.

Para comenzar, cargue el dataset señalado y visualice a través de un `.head` los atributos que posee el dataset.

<i><p align="center">Fiu siendo felicitado por su excelente desempeño en el proyecto de caracterización de datos</p></i>
<p align="center">
  <img src="https://media-front.elmostrador.cl/2023/09/A_UNO_1506411_2440e.jpg">
</p>

In [14]:
import pandas as pd
import numpy as np
from datetime import datetime

df = pd.read_csv('sales.csv')
df['date'] = pd.to_datetime(df['date'])

df.head()

,id,date,city,lat,long,pop,shop,brand,container,capacity,price,quantity
0,0,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,glass,500ml,0.96,13280
1,1,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,plastic,1.5lt,2.86,6727
2,2,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,can,330ml,0.87,9848
3,3,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,adult-cola,glass,500ml,1.00,20050
4,4,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,adult-cola,can,330ml,0.39,25696


## 1.1 Generando un Baseline (0.5 puntos)

<p align="center">
  <img src="https://media.tenor.com/O-lan6TkadUAAAAC/what-i-wnna-do-after-a-baseline.gif">
</p>

Antes de entrenar un algoritmo, usted recuerda los apuntes de su magíster en ciencia de datos y recuerda que debe seguir una serie de *buenas prácticas* para entrenar correcta y debidamente su modelo. Después de un par de vueltas, llega a las siguientes tareas:

1. Separe los datos en conjuntos de train (70%), validation (20%) y test (10%). Fije una semilla para controlar la aleatoriedad.
2. Implemente un `FunctionTransformer` para extraer el día, mes y año de la variable `date`. Guarde estas variables en el formato categorical de pandas.
3. Implemente un `ColumnTransformer` para procesar de manera adecuada los datos numéricos y categóricos. Use `OneHotEncoder` para las variables categóricas.
4. Guarde los pasos anteriores en un `Pipeline`, dejando como último paso el regresor `DummyRegressor` para generar predicciones en base a promedios.
5. Entrene el pipeline anterior y reporte la métrica `mean_absolute_error` sobre los datos de validación. ¿Cómo se interpreta esta métrica para el contexto del negocio?
6. Finalmente, vuelva a entrenar el `Pipeline` pero esta vez usando `XGBRegressor` como modelo **utilizando los parámetros por default**. ¿Cómo cambia el MAE al implementar este algoritmo? ¿Es mejor o peor que el `DummyRegressor`?
7. Guarde ambos modelos en un archivo .pkl (uno cada uno)

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder,StandardScaler,FunctionTransformer,MinMaxScaler
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
import joblib
from sklearn_pandas import DataFrameMapper

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score


SEED = 123

In [16]:
X = df.drop("quantity",axis=1)
y = df["quantity"]

In [17]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=SEED)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.33333, random_state=SEED)

del X_temp
del y_temp
def date2dmy(df):
    df['day'] = df['date'].dt.day.astype('category')
    df['month'] = df['date'].dt.month.astype('category')
    df['year'] = df['date'].dt.year.astype('category')
    return df[['day', 'month', 'year']]

date_transformer = FunctionTransformer(date2dmy, validate=False)
numerical_features = X_train.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X_train.select_dtypes(include=['object']).columns

preprocessor = ColumnTransformer(transformers=[('date', date_transformer, ['date']),
                                               ('num', StandardScaler(), numerical_features),
                                               ('cat',OneHotEncoder(handle_unknown='ignore',sparse_output=False), categorical_features)])
preprocessor.set_output(transform='pandas')

pipe_dummy = Pipeline(steps=[('preprocessor', preprocessor),
                             ('regressor', DummyRegressor())])
display(pipe_dummy)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_function_transformer.py:343: UserWarning: With transform="pandas", `func` should return a DataFrame to follow the set_output API.
  warnings.warn(


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('date',
                                                  FunctionTransformer(func=<function date2dmy at 0x7d42711e37f0>),
                                                  ['date']),
                                                 ('num', StandardScaler(),
                                                  Index(['id', 'lat', 'long', 'pop', 'price'], dtype='object')),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  Index(['city', 'shop', 'brand', 'container', 'capacity'], dtype='object'))])),
                ('regressor', DummyRegressor())])

In [18]:
pipe_dummy.fit(X_train, y_train)

y_val_pred = pipe_dummy.predict(X_val)

mae = mean_absolute_error(y_val, y_val_pred)
print(f'MAE del conjunto val para dummy: {mae}')

MAE del conjunto val para dummy: 13254.740836794412


In [19]:
pipe_xgboost = Pipeline(steps=[('preprocessor', preprocessor),
                               ('regressor', XGBRegressor(enable_categorical = True,random_state=SEED))])


pipe_xgboost.fit(X_train, y_train)

y_val_pred_xgboost = pipe_xgboost.predict(X_val)

mae_xgboost = mean_absolute_error(y_val, y_val_pred_xgboost)
print(f'MAE del conjunto val para XGBoost: {mae_xgboost}')


MAE del conjunto val para XGBoost: 2571.7737574535754


In [20]:
joblib.dump(pipe_dummy, 'pipe_dummy.pkl')
joblib.dump(pipe_dummy.named_steps['regressor'], 'model_dummy.pkl')
joblib.dump(pipe_xgboost, 'pipe_xgboost.pkl')
joblib.dump(pipe_xgboost.named_steps['regressor'], 'model_xgboost.pkl')

['model_xgboost.pkl']

Es posible observar que para los regresores implementados, el MAE varía. Para el Regresor dummy, el valor MAE obtenido en el conjunto de validación es de 13254.74, mientras que para el siguiente regresor XGBoost sin relaciones, el valor desciende a 2571.77. Considerando que un valor menor de MAE es mejor, el regresor que entrega mejores resultados es XGBoost Regressor.

## 1.2 Forzando relaciones entre parámetros con XGBoost (1.0 puntos)

<p align="center">
  <img src="https://64.media.tumblr.com/14cc45f9610a6ee341a45fd0d68f4dde/20d11b36022bca7b-bf/s640x960/67ab1db12ff73a530f649ac455c000945d99c0d6.gif">
</p>

Un colega aficionado a la economía le *sopla* que la demanda guarda una relación inversa con el precio del producto. Motivado para impresionar al querido corpóreo, se propone hacer uso de esta información para mejorar su modelo.

Vuelva a entrenar el `Pipeline`, pero esta vez forzando una relación monótona negativa entre el precio y la cantidad. Luego, vuelva a reportar el `MAE` sobre el conjunto de validación. ¿Cómo cambia el error al incluir esta relación? ¿Tenía razón su amigo?

Nuevamente, guarde su modelo en un archivo .pkl

Nota: Para realizar esta parte, debe apoyarse en la siguiente <a href = https://xgboost.readthedocs.io/en/stable/tutorials/monotonic.html>documentación</a>.

Hint: Para implementar el constraint, se le sugiere hacerlo especificando el nombre de la variable. De ser así, probablemente le sea útil **mantener el formato de pandas** antes del step de entrenamiento.

In [21]:
pipe_xgboost_const = Pipeline(steps=[('preprocessor', preprocessor),
                                     ('regressor', XGBRegressor(random_state=SEED,
                                                                monotone_constraints={"num__price": -1},
                                                                enable_categorical = True,))])
X_transformed = preprocessor.transform(X_train)
display(X_transformed)

pipe_xgboost_const.fit(X_train, y_train)
y_val_pred_xgboost_const = pipe_xgboost_const.predict(X_val)

mae_xgboost_const = mean_absolute_error(y_val, y_val_pred_xgboost_const)
print(f'MAE del conjunto val para XGBoost con relaciones: {mae_xgboost_const}')

,date__day,date__month,date__year,num__id,num__lat,num__long,num__pop,num__price,cat__city_Athens,cat__city_Irakleion,...,cat__brand_gazoza,cat__brand_kinder-cola,cat__brand_lemon-boost,cat__brand_orange-power,cat__container_can,cat__container_glass,cat__container_plastic,cat__capacity_1.5lt,cat__capacity_330ml,cat__capacity_500ml
3785,31,7,2015,0.020555,0.810835,-0.781462,-0.915526,-0.435075,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
6199,30,11,2017,1.147845,-0.196706,0.414637,1.338319,0.369038,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
5432,28,2,2017,0.789203,-0.035621,-1.410704,-0.800417,-0.057386,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2299,28,2,2014,-0.671963,-0.207040,0.387798,1.348347,0.734544,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
4568,30,4,2016,0.383324,1.422810,-0.308728,-0.012636,0.113184,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5218,30,11,2016,0.689682,1.422810,-0.308728,-0.012636,0.722361,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
4060,31,10,2015,0.148510,0.810835,-0.781462,-0.915526,-0.800582,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
1346,30,4,2013,-1.116367,-1.808575,1.729167,-0.942616,-0.569094,0.0,1.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3454,31,3,2015,-0.134459,-0.207040,0.387798,1.344193,2.281854,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0


MAE del conjunto val para XGBoost con relaciones: 2721.9707533940623


Para el regresor XGBoost con relaciones, el MAE asciende a 2721.97.

## 1.3 Optimización de Hiperparámetros con Optuna (2.0 puntos)

<p align="center">
  <img src="https://media.tenor.com/fmNdyGN4z5kAAAAi/hacking-lucy.gif">
</p>

Luego de presentarle sus resultados, Fiu le pregunta si es posible mejorar *aun más* su modelo. En particular, le comenta de la optimización de hiperparámetros con metodologías bayesianas a través del paquete `optuna`. Como usted es un aficionado al entrenamiento de modelos de ML, se propone implementar la descabellada idea de su jefe.

A partir de la mejor configuración obtenida en la sección anterior, utilice `optuna` para optimizar sus hiperparámetros. En particular, se le pide:

- Fijar una semilla en las instancias necesarias para garantizar la reproducibilidad de resultados
- Utilice `TPESampler` como método de muestreo
- De `XGBRegressor`, optimice los siguientes hiperparámetros:
    - `learning_rate` buscando valores flotantes en el rango (0.001, 0.1)
    - `n_estimators` buscando valores enteros en el rango (50, 1000)
    - `max_depth` buscando valores enteros en el rango (3, 10)
    - `max_leaves` buscando valores enteros en el rango (0, 100)
    - `min_child_weight` buscando valores enteros en el rango (1, 5)
    - `reg_alpha` buscando valores flotantes en el rango (0, 1)
    - `reg_lambda` buscando valores flotantes en el rango (0, 1)
- De `OneHotEncoder`, optimice el hiperparámetro `min_frequency` buscando el mejor valor flotante en el rango (0.0, 1.0)
- Explique cada hiperparámetro y su rol en el modelo. ¿Hacen sentido los rangos de optimización indicados?
- Fije el tiempo de entrenamiento a 5 minutos
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto?
- Guardar su modelo en un archivo .pkl

In [12]:
import optuna
def objective(trial):
    xgb_params = {
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'max_leaves': trial.suggest_int('max_leaves', 0, 100),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 5),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 1),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 1)
    }

    pipe_xgboost_const.named_steps['regressor'].set_params(**xgb_params)
    min_frequency = trial.suggest_float('min_frequency', 0.0, 0.1)
    pipe_xgboost_const.named_steps['preprocessor'].named_transformers_['cat'].set_params(min_frequency=min_frequency)

    pipe_xgboost_const.fit(X_train, y_train)
    y_val_pred_xgboost_optuna = pipe_xgboost_const.predict(X_val)
    mae_xgboost_optuna = mean_absolute_error(y_val, y_val_pred_xgboost_optuna)

    return mae_xgboost_optuna

study = optuna.create_study(sampler=optuna.samplers.TPESampler(), direction='minimize')
study.optimize(objective, timeout=5*60)


[I 2023-11-17 01:12:44,362] A new study created in memory with name: no-name-a756ef6c-e353-4f35-a46f-8449030a3add
[I 2023-11-17 01:12:48,766] Trial 0 finished with value: 2387.0400942438487 and parameters: {'learning_rate': 0.08953832552539143, 'n_estimators': 259, 'max_depth': 7, 'max_leaves': 42, 'min_child_weight': 5, 'reg_alpha': 0.11204195384558835, 'reg_lambda': 0.8760523629344331, 'min_frequency': 0.09007122107970955}. Best is trial 0 with value: 2387.0400942438487.
[I 2023-11-17 01:12:50,919] Trial 1 finished with value: 2343.2589144895414 and parameters: {'learning_rate': 0.0947626456170318, 'n_estimators': 414, 'max_depth': 6, 'max_leaves': 89, 'min_child_weight': 2, 'reg_alpha': 0.18635101600168935, 'reg_lambda': 0.8340821624569345, 'min_frequency': 0.0265811234885852}. Best is trial 1 with value: 2343.2589144895414.
[I 2023-11-17 01:12:52,393] Trial 2 finished with value: 2357.328004485245 and parameters: {'learning_rate': 0.036149055680252215, 'n_estimators': 727, 'max_dep

---
1. `learning_rate`:  valores flotantes en el rango (0.001, 0.1)

> La tasa de aprendizaje controla el tamaño del paso durante el proceso de optimización. Una tasa de aprendizaje más pequeña permite una optimización más precisa pero puede requerir más iteraciones. El rango 0.001, 0.1 es un rango aceptable, pues justamente barre sobre los pasos muy pequeños, y  los pasos considerados grandes.
---

2. `n_estimators`:  valores enteros en el rango (50, 1000)

> Determina el número de rondas de impulso (árboles) en el modelo. Un número más alto de estimadores puede mejorar el rendimiento del modelo, pero hay un equilibrio entre la complejidad del modelo y el tiempo de entrenamiento. El rango (50, 1000) cubre un amplio espectro para encontrar un equilibrio apropiado.

---
3. `max_depth`:  valores enteros en el rango (3, 10)
> Es la profundidad máxima de cada árbol. Un max_depth más alto permite al modelo capturar patrones más complejos en los datos, pero puede causar sobreajuste. El rango (3, 10)es un rango apropiado para encontrar una profundidad que evite el sobreajuste mientras captura los patrones.
----
4. `max_leaves`:  valores enteros en el rango (0, 100)
> controla el número máximo de hojas por árbol. Un valor más alto permite árboles más complejos, sin embargo demasiadas hojas pueden provocar sobreajuste. El rango (0, 100) cubre un amplio espectro de valores.

---
5. `min_child_weight`:  valores enteros en el rango (1, 5)
> Es la suma mínima de peso de instancia necesaria en un hijo. Controla el sobreajuste. El rango (1, 5) es apropiado para encontrar un valor que evite que el modelo sea demasiado sensible a instancias individuales.
---
6. `reg_alpha`:  valores flotantes en el rango (0, 1)
> Término de regularización L1 en los pesos. Ayuda a prevenir el sobreajuste penalizando los pesos grandes. El rango (0, 1) permite ajustar la fuerza de regularización.
---
7. `reg_lambda`:  valores flotantes en el rango (0, 1)
> regularización L2 en los pesos. Al igual que reg_alpha, previene el sobreajuste penalizando los pesos grandes. El rango (0, 1) permite ajustar la fuerza de la regularización.

Los mejores resultados para el estudio son los siguientes:

In [22]:
print(f'N  trials: {len(study.trials)}')
print(f'Mejor trial:')
trial = study.best_trial
print(f'  Valor: {trial.value}')


N  trials: 96
Mejor trial:
  Valor: 2151.2438431041983


In [23]:
best_params = study.best_params
for param in best_params:
    print(f'{param}:{best_params[param]}')

learning_rate:0.054604020600623464
n_estimators:872
max_depth:7
max_leaves:84
min_child_weight:4
reg_alpha:0.07481649665830375
reg_lambda:0.4022717910187951
min_frequency:0.008777017313997482


Reasignamos los parámetros del modelo con los recién encontrados

In [24]:

pipe_xgboost_const.named_steps['regressor'].set_params(**best_params)
preprocessor.named_transformers_['cat'].set_params(min_frequency = best_params['min_frequency'])

pipe_xgboost_const.fit(X_train, y_train)
y_val_pred_xgboost_const = pipe_xgboost_const.predict(X_val)

mae_xgboost_const = mean_absolute_error(y_val, y_val_pred_xgboost_const)
print(f'MAE del conjunto val para XGBoost con optuna: {mae_xgboost_const}')

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [01:17:53] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "min_frequency" } are not used.

  warnings.warn(smsg, UserWarning)


MAE del conjunto val para XGBoost con relaciones: 2151.2438431041983


Los resultados de este modelo son mejores que los modelos anteriores. Esto tiene sentido, pues se han elegido parámetros más óptimos que los por defecto seleccionados en las partes anteriores.

In [25]:
joblib.dump(pipe_xgboost_const, 'pipe_xgboost_mejor.pkl')
joblib.dump(pipe_xgboost_const.named_steps['regressor'], 'model_xgboost_const_mejor.pkl')

['model_xgboost_const_mejor.pkl']

## 1.4 Optimización de Hiperparámetros con Optuna y Prunners (1.7)

<p align="center">
  <img src="https://i.pinimg.com/originals/90/16/f9/9016f919c2259f3d0e8fe465049638a7.gif">
</p>

Después de optimizar el rendimiento de su modelo varias veces, Fiu le pregunta si no es posible optimizar el entrenamiento del modelo en sí mismo. Después de leer un par de post de personas de dudosa reputación en la *deepweb*, usted llega a la conclusión que puede cumplir este objetivo mediante la implementación de **Prunning**.

Vuelva a optimizar los mismos hiperparámetros que la sección pasada, pero esta vez utilizando **Prunning** en la optimización. En particular, usted debe:

- Responder: ¿Qué es prunning? ¿De qué forma debería impactar en el entrenamiento?
- Utilizar `optuna.integration.XGBoostPruningCallback` como método de **Prunning**
- Fijar nuevamente el tiempo de entrenamiento a 5 minutos
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto?
- Guardar su modelo en un archivo .pkl

Nota: Si quieren silenciar los prints obtenidos en el prunning, pueden hacerlo mediante el siguiente comando:

```
optuna.logging.set_verbosity(optuna.logging.WARNING)
```

De implementar la opción anterior, pueden especificar `show_progress_bar = True` en el método `optimize` para *más sabor*.

Hint: Si quieren especificar parámetros del método .fit() del modelo a través del pipeline, pueden hacerlo por medio de la siguiente sintaxis: `pipeline.fit(stepmodelo__parametro = valor)`

Hint2: Este <a href = https://stackoverflow.com/questions/40329576/sklearn-pass-fit-parameters-to-xgboost-in-pipeline>enlace</a> les puede ser de ayuda en su implementación


El pruning es una tecnica de compresion de datos cuyo objetivo es reducir el tamaño de los arboles de decision mediante la eliminación de zonas de los arboles que son redundantes y/o no aportan al momento de clasificación. Esta tecnica tiene mucho sentido utilizarla en este contexto debido a que en escencia, el XGboost es un modelo que emplea arboles de decisión

In [26]:
import xgboost as xgb

def objective(trial):

    pipeline = Pipeline([
        ('preprocessor', preprocessor)
    ])

    X_transformed = pipeline.fit_transform(X_train)
    X_val_trans = pipeline.transform(X_val)

    xgb_params = {
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'max_leaves': trial.suggest_int('max_leaves', 0, 100),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 5),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 1),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 1)
    }

    reg.set_params(**xgb_params)
    min_frequency = trial.suggest_float('min_frequency', 0.0, 0.1)
    pipeline.named_steps['preprocessor'].named_transformers_['cat'].set_params(min_frequency=min_frequency)

    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, 'validation_1-mae')

    reg.fit(X_transformed, y_train,
            eval_metric=['mae'],
            eval_set=[(X_transformed, y_train), (X_val_trans, y_val)],
            callbacks=[pruning_callback])
    y_val_pred_xgboost_optuna = reg.predict(X_val_trans)
    mae_xgboost_optuna = mean_absolute_error(y_val, y_val_pred_xgboost_optuna)

    return mae_xgboost_optuna

reg = XGBRegressor(random_state=SEED,enable_categorical=True)

optuna.logging.set_verbosity(optuna.logging.WARNING)
study = optuna.create_study(direction="minimize")
study.optimize(objective, timeout = 5*60)


[0]	validation_0-mae:13379.63436	validation_1-mae:13205.50066
[1]	validation_0-mae:13327.43241	validation_1-mae:13157.11178
[2]	validation_0-mae:13275.55328	validation_1-mae:13107.78731
[3]	validation_0-mae:13224.43340	validation_1-mae:13060.48373
[4]	validation_0-mae:13172.27836	validation_1-mae:13011.92601
[5]	validation_0-mae:13121.89950	validation_1-mae:12965.21131
[6]	validation_0-mae:13070.88057	validation_1-mae:12917.42916
[7]	validation_0-mae:13019.88851	validation_1-mae:12869.69731
[8]	validation_0-mae:12969.22275	validation_1-mae:12822.28153
[9]	validation_0-mae:12920.34515	validation_1-mae:12776.77110
[10]	validation_0-mae:12870.41152	validation_1-mae:12730.36606
[11]	validation_0-mae:12821.08508	validation_1-mae:12684.54227
[12]	validation_0-mae:12773.60327	validation_1-mae:12639.78456
[13]	validation_0-mae:12724.99641	validation_1-mae:12594.52992
[14]	validation_0-mae:12678.18893	validation_1-mae:12552.24839
[15]	validation_0-mae:12630.31878	validation_1-mae:12507.69197
[1

/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:885: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:885: UserWarning: `callbacks` in `fit` method is deprecated for better compatibility with scikit-learn, use `callbacks` in constructor or`set_params` instead.
  warnings.warn(


[26]	validation_0-mae:12131.44408	validation_1-mae:12046.20771
[27]	validation_0-mae:12088.67479	validation_1-mae:12007.36835
[28]	validation_0-mae:12045.14889	validation_1-mae:11967.47248
[29]	validation_0-mae:12002.30656	validation_1-mae:11927.92450
[30]	validation_0-mae:11959.86112	validation_1-mae:11889.41432
[31]	validation_0-mae:11917.24720	validation_1-mae:11849.92445
[32]	validation_0-mae:11875.58890	validation_1-mae:11812.13575
[33]	validation_0-mae:11833.85803	validation_1-mae:11774.51968
[34]	validation_0-mae:11792.16101	validation_1-mae:11735.33157
[35]	validation_0-mae:11751.03108	validation_1-mae:11697.92096
[36]	validation_0-mae:11710.10690	validation_1-mae:11659.43546
[37]	validation_0-mae:11670.19009	validation_1-mae:11623.05792
[38]	validation_0-mae:11629.66316	validation_1-mae:11585.88494
[39]	validation_0-mae:11589.71213	validation_1-mae:11549.26662
[40]	validation_0-mae:11549.40142	validation_1-mae:11511.98170
[41]	validation_0-mae:11510.64603	validation_1-mae:1147

/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:885: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:885: UserWarning: `callbacks` in `fit` method is deprecated for better compatibility with scikit-learn, use `callbacks` in constructor or`set_params` instead.
  warnings.warn(


[12]	validation_0-mae:13047.92108	validation_1-mae:12904.73252
[13]	validation_0-mae:13018.89900	validation_1-mae:12878.26352
[14]	validation_0-mae:12991.48447	validation_1-mae:12853.21103
[15]	validation_0-mae:12963.27437	validation_1-mae:12827.28926
[16]	validation_0-mae:12936.35659	validation_1-mae:12803.66043
[17]	validation_0-mae:12908.34116	validation_1-mae:12778.62534
[18]	validation_0-mae:12881.33811	validation_1-mae:12753.98352
[19]	validation_0-mae:12853.91665	validation_1-mae:12728.59321
[20]	validation_0-mae:12826.66077	validation_1-mae:12703.66436
[21]	validation_0-mae:12800.65858	validation_1-mae:12680.96032
[22]	validation_0-mae:12773.82254	validation_1-mae:12656.20396
[23]	validation_0-mae:12746.79640	validation_1-mae:12630.90025
[24]	validation_0-mae:12721.91154	validation_1-mae:12609.22648
[25]	validation_0-mae:12696.68383	validation_1-mae:12587.12865
[26]	validation_0-mae:12672.39643	validation_1-mae:12565.21357
[27]	validation_0-mae:12646.58255	validation_1-mae:1254

...

[0]	validation_0-mae:12703.79900	validation_1-mae:12563.82225
[1]	validation_0-mae:12019.23566	validation_1-mae:11920.77102
[2]	validation_0-mae:11390.33328	validation_1-mae:11332.59831
[3]	validation_0-mae:10802.00987	validation_1-mae:10796.57425
[4]	validation_0-mae:10258.46291	validation_1-mae:10297.31691
[5]	validation_0-mae:9750.17490	validation_1-mae:9827.09261
[6]	validation_0-mae:9283.95552	validation_1-mae:9400.90380
[7]	validation_0-mae:8852.56688	validation_1-mae:8997.29526


/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:885: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:885: UserWarning: `callbacks` in `fit` method is deprecated for better compatibility with scikit-learn, use `callbacks` in constructor or`set_params` instead.
  warnings.warn(


[8]	validation_0-mae:8440.73263	validation_1-mae:8630.87634
[9]	validation_0-mae:8059.31038	validation_1-mae:8282.86294
[10]	validation_0-mae:7713.13386	validation_1-mae:7961.29139
[11]	validation_0-mae:7372.67974	validation_1-mae:7654.17665
[12]	validation_0-mae:7074.67369	validation_1-mae:7379.42370
[13]	validation_0-mae:6776.93782	validation_1-mae:7116.11858
[14]	validation_0-mae:6506.09450	validation_1-mae:6868.39829
[15]	validation_0-mae:6235.77558	validation_1-mae:6628.77821
[16]	validation_0-mae:5996.55717	validation_1-mae:6412.83148
[17]	validation_0-mae:5749.07930	validation_1-mae:6176.84385
[18]	validation_0-mae:5534.09398	validation_1-mae:5989.72521
[19]	validation_0-mae:5337.47201	validation_1-mae:5814.68318
[20]	validation_0-mae:5134.08130	validation_1-mae:5631.05033
[21]	validation_0-mae:4958.72333	validation_1-mae:5475.65291
[22]	validation_0-mae:4779.05281	validation_1-mae:5309.46872
[23]	validation_0-mae:4606.93292	validation_1-mae:5154.49632
[24]	validation_0-mae:4460

/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:885: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:885: UserWarning: `callbacks` in `fit` method is deprecated for better compatibility with scikit-learn, use `callbacks` in constructor or`set_params` instead.
  warnings.warn(


[9]	validation_0-mae:6834.67096	validation_1-mae:7161.40128
[10]	validation_0-mae:6463.04858	validation_1-mae:6821.98935
[11]	validation_0-mae:6138.08528	validation_1-mae:6524.41540
[12]	validation_0-mae:5810.79849	validation_1-mae:6232.94971
[13]	validation_0-mae:5491.67680	validation_1-mae:5948.86125
[14]	validation_0-mae:5225.57829	validation_1-mae:5705.85574
[15]	validation_0-mae:4954.61030	validation_1-mae:5453.73873
[16]	validation_0-mae:4719.24417	validation_1-mae:5235.05272
[17]	validation_0-mae:4487.29826	validation_1-mae:5043.20775
[18]	validation_0-mae:4288.14114	validation_1-mae:4863.45223
[19]	validation_0-mae:4104.21567	validation_1-mae:4704.24833
[20]	validation_0-mae:3936.77573	validation_1-mae:4556.82246
[21]	validation_0-mae:3783.60515	validation_1-mae:4424.72510
[22]	validation_0-mae:3643.05323	validation_1-mae:4309.58712
[23]	validation_0-mae:3501.02757	validation_1-mae:4185.11315
[24]	validation_0-mae:3382.67957	validation_1-mae:4085.01609
[25]	validation_0-mae:326

In [27]:
#print(study.best_trial)
best_params = study.best_params
reg.set_params(**best_params)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=True, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.06698014072330198,
             max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=7, max_leaves=67,
             min_child_weight=3, min_frequency=0.024619629108167482,
             missing=nan, monotone_constraints=None, multi_strategy=None,
             n_estimators=1000, n_jobs=None, num_parallel_tree=None, ...)

In [28]:

# Create the pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor)
])

# Assuming you have a DataFrame X
X_transformed = pipeline.fit_transform(X_train)
X_val_trans = pipeline.transform(X_val)

y_val_pred_xgboost_const = reg.predict(X_val_trans)

mae_xgboost_const = mean_absolute_error(y_val, y_val_pred_xgboost_const)
print(f'MAE del conjunto val para XGBoost con optuna y prunning: {mae_xgboost_const}')

MAE del conjunto val para XGBoost con relaciones: 2118.273560962127


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [01:23:12] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "min_frequency" } are not used.

  warnings.warn(smsg, UserWarning)


## 1.5 Visualizaciones (0.5 puntos)

<p align="center">
  <img src="https://media.tenor.com/F-LgB1xTebEAAAAd/look-at-this-graph-nickelback.gif">
</p>


Satisfecho con su trabajo, Fiu le pregunta si es posible generar visualizaciones que permitan entender el entrenamiento de su modelo.

A partir del siguiente <a href = https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/005_visualization.html#visualization>enlace</a>, genere las siguientes visualizaciones:

- Gráfico de historial de optimización
- Gráfico de coordenadas paralelas
- Gráfico de importancia de hiperparámetros

Comente sus resultados: ¿Desde qué *trial* se empiezan a observar mejoras notables en sus resultados? ¿Qué tendencias puede observar a partir del gráfico de coordenadas paralelas? ¿Cuáles son los hiperparámetros con mayor importancia para la optimización de su modelo?

In [31]:
from optuna.visualization import plot_contour
from optuna.visualization import plot_edf
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_rank
from optuna.visualization import plot_slice
from optuna.visualization import plot_timeline




In [32]:
plot_optimization_history(study)

In [33]:
plot_parallel_coordinate(study)

In [34]:
plot_param_importances(study)

con respecto al gráfico de historial del estudio, se puede ver que a partir del trial 3 el MAE empieza a decrecer brúscamente y a partir del trial 10 el MAE comienza a mantenerse relativamente constante. Con respecto a las variables mas importantes, se puede ver que reg alpha, max_leaves y learning rate son las mas importantes. Esto tiene mucho sentido porque max_leaves indica la cantidad de hojas que tiene cada arbol estimador y la forma en que este modelo realiza su trabajo depende directamente de la cantidad de hojas de cada arbol. Por otra parte el learning rate ajusta que tan rapido el modelo aprende durante el entrenamiento, por lo que el resultado es consistente con lo que se ve. Finalmente reg_alpha es la mas importante de todas por un gran margen y esto se puede deber a que este parámetro penaliza aquellos features que incrementan la funcion de costos, es decir, este parámetro afecta fuertemente el funcionamiento interno del modelo, por lo que hace sentido que sea la que mas importancia tiene, por mucho.

Con respecto al gráfico de coordenadas paralelas se puede ver que los MAEs mas bajos estan intimamente relacionados con learning rates entre el rango 0.06 y 0.08, profundidades de arboles ya sea de 4 o de 7, numero máximo de hojas superiores a 60, peso mínimo de nodos hijos ya sea de 1 o de 3, sobre 750 estimadores, un reg_alpha entre 0.3 y 0.5 y finalmente un reg_alpha de 0.8 o mas.

## 1.6 Síntesis de resultados (0.3)

Finalmente, genere una tabla resumen del MAE obtenido en los 5 modelos entrenados (desde Baseline hasta XGBoost con Constraints, Optuna y Prunning) y compare sus resultados. ¿Qué modelo obtiene el mejor rendimiento?

Por último, cargue el mejor modelo, prediga sobre el conjunto de test y reporte su MAE. ¿Existen diferencias con respecto a las métricas obtenidas en el conjunto de validación? ¿Porqué puede ocurrir esto?

In [41]:
data = {'Modelo': ['Baseline (dummy)', 'XGBoost (default)', 'XGBoost con relaciones', 'Optuna', 'Prunning'],
        'MAE': [13254.7408, 2571.7737, 2721.9707, 2151.2438, 2118.2735]}

log_modelos = pd.DataFrame(data)
display(log_modelos)

,Modelo,MAE
0,Baseline (dummy),13254.7408
1,XGBoost (default),2571.7737
2,XGBoost con relaciones,2721.9707
3,Optuna,2151.2438
4,Prunning,2118.2735


In [44]:
y_val_pred_xgboost_const_final = reg.predict(pipeline.transform(X_test))
mae_xgboost_const_final = mean_absolute_error(y_test, y_val_pred_xgboost_const_final)
print(f'MAE del conjunto test para XGBoost con optuna y prunning: {mae_xgboost_const_final}')

MAE del conjunto test para XGBoost con optuna y prunning: 2114.3620117842033


El resultado obtenido es aun mejor que cualquiera de los MAEs obtenidos anteriormente, aun cuando fue un conjunto de test con datos que el modelo nunca vio. Sin embargo, esta ocurrencia es poco usual en los modelos y la norma general es que test sea ligeramente peor que validación. Esto puede ocurrir porque la idea detras de tener un conjunto de test es que ninguno de esos datos paso por el modelo en ningun momento, por lo que el modelo idealmente clasifica (o estima) en base a lo que ha aprendido y logrado generalizar. Un mal rendimiento en un conjunto de test implicaría que el modelo esta sobreajustado y esta perdiendo capacidad de generalizar

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media.tenor.com/8CT1AXElF_cAAAAC/gojo-satoru.gif">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>